## Setup

Install dependencies

In [ ]:
!pip install madml

Import packages for run

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from madml.ml.splitters import BootstrappedLeaveClusterOut
from madml.models.space import distance_model
from madml.models.combine import domain_model
from madml.models.uq import calibration_model
from madml.ml.assessment import nested_cv
from madml import datasets

## Load data
There are a set of datasets available. You can load any of them with the name given by the following command.

In [ ]:
datasets.list_data()

Any of the supported data can be loaded in a standard manner. You are capable of loading your own data instead of any of the supported datasest if needed.

In [ ]:
    data = datasets.load('diffusion')
    X = data['data']
    y = data['target']
    g = data['class_name']

## Build model
We define three model types: uncertanty quantification, distance, and regression model. If we want uncertainty quantification, the regression model must be an ensmble model (e.g. random forest, bagged LASSO, et cetera).

We start with a distance model.

In [ ]:
ds_model = distance_model(dist='kde')

Now we add a polynomial uncertaty quantifiction model. The number of arguments for te argument params defines the degree of the polynomial, and their values are the inital guesses to the optimizer.

In [ ]:
uq_model = calibration_model(params=[0.0, 1.0])

Now we define the regression model. The regression model must be a gridserach object with a pipeline. The example here does not iterate over folds for hyperparamter optimization, but it can be modified to do so.

In [ ]:
# ML
scale = StandardScaler()
model = RandomForestRegressor()

# The grid for grid search
grid = {}
grid['model__n_estimators'] = [100]

# The machine learning pipeline
pipe = Pipeline(steps=[
                        ('scaler', scale),
                        ('model', model),
                        ])

# The gridsearch model
gs_model = GridSearchCV(
                        pipe,
                        grid,
                        cv=((slice(None), slice(None)),),  # No splits
                        )

# Building the splits
Here comes the fun part. The performance of a model on a test set depends on many things. We want to guard against using predictions on data that are sampled dissimilarly to the data used for training. First, we build splits where test data are sampled similarly to training data. We give these splits the special name of "calibration" so that we only use this kind of splitter for our uncertainty quantification model.

In [ ]:
n_repeats = 2  # The number of time to repeat splits
splits = [('calibration', RepeatedKFold(n_repeats=n_repeats))]

How we need to tell the model what data are dissimilar. We use come pre-clustering and split data accordingly. Here, we do 2 and 3 cluster and use agglomerative clustering.

In [ ]:
for i in [2, 3]:

    # Cluster Splits
    top_split = BootstrappedLeaveClusterOut(
                                            AgglomerativeClustering,
                                            n_repeats=n_repeats,
                                            n_clusters=i
                                            )

    splits.append(('agglo_{}'.format(i), top_split))

# Assessing and Fitting the Model
Now, we assess the model through cross valication and then fit a final model on all data that can be used by a researcher. The assessment of the model is saved in a directory of the user's choice.

In [ ]:
model = domain_model(gs_model, ds_model, uq_model, splits)
cv = nested_cv(X, y, g, model, splits, save='./run')
_, model = cv.assess()

# Example of Model Use
Our assessment returns a model. We can also use dill to load the saved model in the ./run/model/model.dill. Here, we predict on the features used to build the model.

In [ ]:
df = model.predict(X)
print(df)